In [23]:
# Yerbol Aussat (SID: 20698564)
# CS698, Assignment 3
# Logistic Regression (One vs All)
import numpy as np

In [24]:
# unpickle the data
def unpickle(file):
    import cPickle
    with open(file, 'rb') as fo:
        dict = cPickle.load(fo)
    return dict

In [25]:
#import training data
batch1_train = unpickle("cifar-10-batches-py/data_batch_1")
trn_data = batch1_train['data']
trn_labels = np.array(batch1_train['labels'])

In [26]:
# import testing data
batch_test = unpickle("cifar-10-batches-py/test_batch")
test_data = batch_test['data']
test_labels = np.array(batch_test['labels'])

In [27]:
def sigmoid(x, w):                                                        
    z = np.dot(x, w)
    return 1.0 / (1.0 + np.exp(-z)) 

In [28]:
def gradient(x, y, w, reg_const, cl1, cl2):   
    # indices of positive and negative labels
    y_pos = np.where(y == cl1) # locations of y = +1
    y_neg = np.where(y == cl2) # locations of y = -1
    
    # number of positive and negative smaples
    n_pos = len(y_pos[0])
    n_neg = len(y_neg[0])
    p = sigmoid(x, w)
    # gradient:
    g = 1.0/n_pos * np.dot(x[y_pos].T, (p[y_pos] - np.ones((n_pos, 1)) )) + 1.0/n_neg * np.dot(x[y_neg].T, (p[y_neg])) + 2.0 *reg_const * w
    return g
                                       

In [29]:
def hessian(x, y, w, reg_const, cl1, cl2):  
    # indices of positive and negative labels
    y_pos = np.where(y == cl1) # locations of y = +1
    y_neg = np.where(y == cl2) # locations of y = -1
    
    # number of positive and negative smaples
    n_pos = len(y_pos[0])
    n_neg = len(y_neg[0])
    p = sigmoid(x, w)
        
    # Hessian:
    H = 1.0/n_pos * x[y_pos].T.dot(np.diagflat(p[y_pos])).dot(np.diagflat(np.ones((n_pos, 1)) - p[y_pos])).dot(x[y_pos]) + 1.0/n_neg * x[y_neg].T.dot(np.diagflat(p[y_neg])).dot(np.diagflat(np.ones((n_neg, 1)) - p[y_neg])).dot(x[y_neg]) + 2.0 * reg_const * np.identity(len(w))
    
    return H
    

In [30]:
from copy import deepcopy

def Newton(X_train, y_train, reg_const, cl1, cl2):
    w = np.zeros((3072, 1))
    max_pass = 50
    
    iteration = 0
    
    while True:
        print "    iteration", iteration, ": ",
        
        w_prev = deepcopy(w)
        g = gradient(X_train, y_train, w, reg_const, cl1, cl2)
        H = hessian(X_train, y_train, w, reg_const, cl1, cl2)
        w = w_prev - np.linalg.solve(H, g)

        print np.linalg.norm(w - w_prev)
        iteration += 1 
        
        if (np.linalg.norm(w - w_prev) < 10**-4) or (iteration >= 50):
            return w           
    return w

In [35]:
# Function calculates percent error of  logistic regression algorithm
def percent_error(trn_data, trn_labels, test_data, test_labels, lambdas):
    for i in range(len(lambdas)):   
        reg_const = lambdas[i]
        print "\nONE VS ONE: lambda =", reg_const
        
        # Training kernel logistic regression algorithm for the value of lambda
        print "Training logistic regression for lambda =", reg_const
        w, h = 10, 10
        w_vectors = [[0 for x in range(w)] for y in range(h)] 
        for i in range(10):
            for j in range(i+1, 10):
                print "\n  Classes", i, j
                w_ij = Newton(trn_data, trn_labels, reg_const, i, j)
                w_vectors[i][j] = w_ij            
            
        # Testing kernel logistic regression algorithm for lambda = 1
        print "Testing logistic regression for lambda =", reg_const
        incorrect = 0
        for sample_i in range(len(trn_labels)):
            if sample_i%1000 == 0:
                print "  Number of predictions done:", sample_i 
            votes = [0 for vote_i in range(10)]
            x = test_data[sample_i]
            for i in range(10):
                for j in range(i+1, 10):
                    w_ij = w_vectors[i][j]
                    p0 = sigmoid(x, w_ij)
                    if (p0 >= 0.5):
                        votes[i]+=1
                    else:
                        votes[j]+=1
                    prediction = np.argmax(votes)
            if prediction != test_labels[sample_i]:
                incorrect+=1
        print "\nOne vs One: lambda =", reg_const, ":"
        print "  Num Incorrectly Classified:", incorrect
        print "  Percent Error:", 1.0*incorrect / len(test_labels)
        print "\n", '*'*40, '\n'

In [41]:
#Finding percent error for lambdas = 0.1, 1, 10
percent_error(trn_data, trn_labels, test_data, test_labels, [0.1, 1, 10])

 
ONE VS ONE: lambda = 0.1
Training logistic regression for lambda = 0.1

  Classes 0 1
    iteration 0 :  0.43932166801
    iteration 1 :  0.135854622265
    iteration 2 :  0.159420950382
    iteration 3 :  0.186095502161
    iteration 4 :  0.145924468214
    iteration 5 :  0.119264733641
    iteration 6 :  0.0947692455959
    iteration 7 :  0.0597220625462
    iteration 8 :  0.0311248022153
    iteration 9 :  0.00830216051718
    iteration 10 :  0.00108570331929
    iteration 11 :  2.76595552544e-05

  Classes 0 2
    iteration 0 :  0.559538489258
    iteration 1 :  0.175489586064
    iteration 2 :  0.21017654842
    iteration 3 :  0.223352148876
    iteration 4 :  0.185980307979
    iteration 5 :  0.137876533708
    iteration 6 :  0.0886509178119
    iteration 7 :  0.047020818429
    iteration 8 :  0.0152243654536
    iteration 9 :  0.00180397731917
    iteration 10 :  4.98714715096e-05

  Classes 0 3
    iteration 0 :  0.518117440485
    iteration 1 :  0.16693792684
    iteration 2

    iteration 9 :  2.12792447763e-05

  Classes 2 5
    iteration 0 :  0.603534990089
    iteration 1 :  0.180394879432
    iteration 2 :  0.21031658977
    iteration 3 :  0.244310977483
    iteration 4 :  0.209668712182
    iteration 5 :  0.140142175844
    iteration 6 :  0.0695244187471
    iteration 7 :  0.0280830926502
    iteration 8 :  0.00499578614976
    iteration 9 :  0.000235474255471
    iteration 10 :  9.57065527842e-07

  Classes 2 6
    iteration 0 :  0.561891298523
    iteration 1 :  0.174337332481
    iteration 2 :  0.205750755612
    iteration 3 :  0.232494981809
    iteration 4 :  0.193875493523
    iteration 5 :  0.140536742429
    iteration 6 :  0.0860526381122
    iteration 7 :  0.0377402218786
    iteration 8 :  0.00939682207204
    iteration 9 :  0.000884172844874
    iteration 10 :  4.82330286338e-05

  Classes 2 7
    iteration 0 :  0.570370588965
    iteration 1 :  0.169686380173
    iteration 2 :  0.215558704898
    iteration 3 :  0.232032385166
    iteration

    iteration 4 :  0.145547347244
    iteration 5 :  0.113926791616
    iteration 6 :  0.0880770455864
    iteration 7 :  0.0672568348458
    iteration 8 :  0.0433505109842
    iteration 9 :  0.0194478355111
    iteration 10 :  0.00480162552237
    iteration 11 :  0.000353612309773
    iteration 12 :  3.6643101477e-06

  Classes 6 7
    iteration 0 :  0.489769938649
    iteration 1 :  0.143815401362
    iteration 2 :  0.191754645706
    iteration 3 :  0.199927730344
    iteration 4 :  0.167785116491
    iteration 5 :  0.128764490161
    iteration 6 :  0.098905080432
    iteration 7 :  0.0707376479839
    iteration 8 :  0.0392981977367
    iteration 9 :  0.013829619193
    iteration 10 :  0.00202898189576
    iteration 11 :  8.11304825741e-05

  Classes 6 8
    iteration 0 :  0.423229084794
    iteration 1 :  0.139047836625
    iteration 2 :  0.161772186402
    iteration 3 :  0.166833507612
    iteration 4 :  0.146176937503
    iteration 5 :  0.111175297955
    iteration 6 :  0.08264997

    iteration 8 :  0.000442468614609
    iteration 9 :  5.74376743568e-06

  Classes 1 6
    iteration 0 :  0.178899032674
    iteration 1 :  0.0650800390971
    iteration 2 :  0.0648370647605
    iteration 3 :  0.0661592803397
    iteration 4 :  0.0543450991434
    iteration 5 :  0.0367089807564
    iteration 6 :  0.0246511230727
    iteration 7 :  0.00943745002015
    iteration 8 :  0.00175836404881
    iteration 9 :  8.59534500942e-05

  Classes 1 7
    iteration 0 :  0.201883016052
    iteration 1 :  0.0710121197505
    iteration 2 :  0.0731515444191
    iteration 3 :  0.0707952289944
    iteration 4 :  0.0563347627932
    iteration 5 :  0.0388684362423
    iteration 6 :  0.0213054209315
    iteration 7 :  0.00610666643365
    iteration 8 :  0.000664739484212
    iteration 9 :  1.27782069081e-05

  Classes 1 8
    iteration 0 :  0.204020388856
    iteration 1 :  0.0747077996945
    iteration 2 :  0.0769394017188
    iteration 3 :  0.069868984129
    iteration 4 :  0.0536505994442
 

    iteration 0 :  0.200650497935
    iteration 1 :  0.0717861333001
    iteration 2 :  0.0817601946934
    iteration 3 :  0.0697028956328
    iteration 4 :  0.0558111536069
    iteration 5 :  0.0398937626729
    iteration 6 :  0.0225312018834
    iteration 7 :  0.00778336587037
    iteration 8 :  0.00112700349207
    iteration 9 :  3.96487078852e-05

  Classes 5 9
    iteration 0 :  0.188691949189
    iteration 1 :  0.0651882467854
    iteration 2 :  0.0715866847617
    iteration 3 :  0.0645951742462
    iteration 4 :  0.0512559462348
    iteration 5 :  0.0386841212436
    iteration 6 :  0.0233193458409
    iteration 7 :  0.0088774931758
    iteration 8 :  0.00142049539515
    iteration 9 :  4.44099117856e-05

  Classes 6 7
    iteration 0 :  0.208736690341
    iteration 1 :  0.0758633486871
    iteration 2 :  0.0850182488267
    iteration 3 :  0.0740898676574
    iteration 4 :  0.0566278799503
    iteration 5 :  0.0391761021016
    iteration 6 :  0.0199478893451
    iteration 7 :  0.

    iteration 1 :  0.0292261531803
    iteration 2 :  0.0219941391779
    iteration 3 :  0.00871626633071
    iteration 4 :  0.00124211864092
    iteration 5 :  2.82635782918e-05

  Classes 2 4
    iteration 0 :  0.0940047907885
    iteration 1 :  0.030069951478
    iteration 2 :  0.0178917217869
    iteration 3 :  0.00444914153414
    iteration 4 :  0.000275714032497
    iteration 5 :  1.52883305543e-06

  Classes 2 5
    iteration 0 :  0.0899665880983
    iteration 1 :  0.0305719038163
    iteration 2 :  0.0225330968307
    iteration 3 :  0.0080035934344
    iteration 4 :  0.00095069865591
    iteration 5 :  1.62264728434e-05

  Classes 2 6
    iteration 0 :  0.0870947355697
    iteration 1 :  0.0309794164969
    iteration 2 :  0.0256531228976
    iteration 3 :  0.0114044848375
    iteration 4 :  0.00221233375871
    iteration 5 :  9.65287964362e-05

  Classes 2 7
    iteration 0 :  0.0845682303871
    iteration 1 :  0.0308745511878
    iteration 2 :  0.0250050726908
    iteration 3 